In [1]:
#Extract (from only Sloane records, no indexs) all catnum where place or person name (or both) mentioned with 
#page numbers<pb>, <div2>, and text within <div3>
#There are div3s with no catnum.
#Last modified April 2021
from lxml import etree
import re
import csv
import pandas as pd

In [3]:
path = '/Users/deborahleem/Documents/1_PhD/XMLSloane2021/output_person_place_misc_cleaned_up_25042021.xml'
doc = etree.parse(open(path))
root = doc.getroot()

In [4]:
def remove_element(el):
    parent = el.getparent()
    if el.tail:
        prev = el.getprevious()
        if prev:
            prev.tail = (prev.tail or '') + el.tail
        else:
            parent.text = (parent.text or '') + el.tail
    parent.remove(el)

# Remove every <add rend="del"> and <add rend="pencil"> from document before the rest of analysis
for e in list(root.iterfind('.//{*}add')):
    if ('rend' in e.attrib) and (e.attrib['rend'] == 'del' or e.attrib['rend'] == 'pencil'):
        #print('removing', etree.tostring(e, pretty_print=True))
        if e.text == '-1861' or e.text == '2094.' or e.text == '-382.':
            continue
        remove_element(e)

/Users/deborahleem/scripts/venvs/deb_1.0/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  """


In [5]:
part_excluded = {
  "EPF108491027v": ["2108","2109","2110","2111"], # "Miscellanies"
  "EPF108491177v": ["1","2"], # "Antiquities"
  "EPF108491277": ["57"], # "Mathematical"
}

excluded = {
  "EPF108491178v": ["1","2","3","4"], # "Antiquities"
  "EPF108491179v": ["4","5","6","7","8","1"], # "Antiquities"
  "EPF108491180v": ["2","3"], # "Antiquities"
  "EPF108491181v": ["4","5"], # "Antiquities"
  "EPF108491182v": ["1","2","3","4","5"], # "Antiquities"
  "EPF108491183v": ["6","7","8","9","10","11","12"], # "Antiquities"
  "EPF108491184v": ["13","14","1","2","3","4","5","1"], # "Antiquities"
  "EPF108491186v": ["1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19"], # "Antiquities"
  "EPF108491187v": ["20","21","22","23","24","25","26","27","28","29","30","31","32","33","34","35","36","37","38","39"], # "Antiquities"
  "EPF108491188v": ["40","41","42","43","44","45","46","47","48","1","2","3","4","5","6","7","8","9"], # "Antiquities"
  "EPF108491189v": ["10","11","12","1","2","3","4","5","6"], # "Antiquities"
  "EPF108491190v": ["7","8","9","10","11"], # "Antiquities"
}
#last two entries on EPF108491271 wrapped in div3 are not Sloane, Pictures &c.

In [6]:
def find_div2_label(element):
    while element is not None:
        if element.tag == '{http://www.tei-c.org/ns/1.0}div2':
            label = element.find('{*}label')
            if label is not None:
                return ''.join(label.itertext()).strip()
            return
        element = element.getparent()

In [7]:
def find_enclosing_div3(element):
    while element is not None:
        if element.tag == '{http://www.tei-c.org/ns/1.0}div3':
            return element
        element = element.getparent()
    
def find_pb_xml_id(element):
    element = find_enclosing_div3(element)
    while element is not None:
        if element.tag == '{http://www.tei-c.org/ns/1.0}pb':
            return element.attrib['{http://www.w3.org/XML/1998/namespace}id']
        element = element.getprevious()

In [8]:
def cleanup_catnum(text):
    #removing from beginning and end:
    # \s spaces/tabs/newlines
    # \- dashes
    # . dots
    # ? question marks
    # _ underscores
    # ✓
    text = re.sub(r'^[ ✓\t\n.\-_?]+', "", text)
    text = re.sub(r'[ ✓\t\n.\-_?]+$', "", text)
    text = re.sub(r'[ ✓\t\n.\-_?]+\*', "*", text)
    text = text.replace('. ', ' ')
    return text

In [9]:
# load the json dicts for idmap (placename)
import json

with open('place_id_map.json') as json_file:
    idmap = json.load(json_file)

with open('person_id_map.json') as json_file:
    idmap_person = json.load(json_file)

In [10]:
idmap_person[]

SyntaxError: invalid syntax (<ipython-input-10-c7e3293b026f>, line 1)

In [11]:
[x for x in idmap_person.keys() if x.lower().startswith('earl')]

['Earl of Sandwich',
 'Earl of Oxford ',
 'Earl Marshal ',
 'Earl of Arundell',
 'Earl of Essex',
 'Earl Abercorn',
 'Earl of Abbington']

In [12]:
#ignore non-Sloane catnums and index <label type="index">. End of index </div2>
people_csv_data = []

# labels = set()

for div3 in root.iter("{*}div3"):
#     import pdb; pdb.set_trace()
    persons = list(div3.iter('{*}persName'))
    places = list(div3.iter('{*}placeName'))
    catnums = list(div3.iter('{*}catnum'))
    clean_catnums = [cleanup_catnum(''.join(catnum.itertext())) for catnum in catnums]
    
    page_id = find_pb_xml_id(div3)
    label = find_div2_label(div3)
    
#     labels.add(label)
    if label.startswith('Index'):
        continue
    
    if len(persons) > 0 or len(places) > 0:
        if page_id in excluded:
            continue

#         This is an individual row to be saved as a dict
        res = {}

        res["div2_label"] = label
        div3_text = ''.join(div3.itertext())

        person_id_res = []
        person_res = []
        for person in persons:
            name = re.sub(r'\s+', " ", ''.join(person.itertext()))
            name = re.sub(r'^\s+', '', name)
            name = re.sub(r'\s+$', '', name)  
            person_res.append(name)
            person_id_res.append(idmap_person.get(name))

        res["person"] = person_res
        res["person_id"] = person_id_res


        places_res = []
        place_id_res = []
        for place in places:
            name = re.sub(r'\s+', " ", ''.join(place.itertext()))
            name = re.sub(r'^\s+', '', name)
            name = re.sub(r'\s+$', '', name)   
            places_res.append(name)
            place_id_res.append(idmap.get(name.lower(), None))

        res["place"] = places_res
        res['place_id'] = place_id_res

        catnum_res = []
        
#         Clean the catnums
        for name in clean_catnums:
            if not name:
                name = ''
            catnum_res.append(name)
            
        if not catnum_res:
            catnum_res = ['']
        res["catnum"] = catnum_res
        people_csv_data.append(res)

# print(labels)

In [13]:
idmap

{'aquisgrana': '#Aachen',
 'achies cliffe': '#Achres',
 'achres a sea cliffe in the county of donegall in ireland': '#Achres',
 'sands near acon': '#Acon',
 'aethiopia': '#Aethiopia',
 '√üthiopish': '#Aethiopia',
 'æthiopish': '#Aethiopia',
 'africa': '#Africa',
 'agrafee': '#Agrafee',
 'alexandria': '#Alexandria',
 'alps': '#Alps',
 'althorp in northamptonshire': '#Althorp',
 'althorpe': '#Althorp',
 'althrop': '#Althorp',
 'amboina': '#AmbonIsland',
 'america': '#America',
 'amesbury on salisbury plain in wiltshire': '#Amesbury',
 'amsterdam': '#Amsterdam',
 'amster= dam': '#Amsterdam',
 'andes': '#Andes',
 'anglesey': '#Anglesey',
 'angola': '#Angola',
 'antego': '#Antigua',
 'antegoa': '#Antigua',
 'antegua': '#Antigua',
 'antigua': '#Antigua',
 'antwerp': '#Antwerp',
 'arabia': '#Arabia',
 'arkindale in yorkshire': '#Arkendale',
 'ashante': '#Ashante',
 'ashanta': '#Ashante',
 'ashover church yard derbyshire at the w, end of the church close to the\n              foundation': '#As

In [14]:
idmap.get('East Indies'.lower())

'#East_Indies'

In [15]:
# Create the dataframe and populate based on the dictionary

df = pd.DataFrame({'div2_label': [], 'person': [], 'person_id': [], 'place': [], 'place_id': [], 'catnum': []})
for row in people_csv_data:
    df = df.append({'div2_label': row['div2_label'], 'person': str(row['person']), 'person_id': str(row['person_id']),'place': str(row['place']), 'place_id': str(row['place_id']), 'catnum': str(row['catnum'])}, ignore_index=True)
df.head()

,div2_label,person,person_id,place,place_id,catnum
0,Miscellanies.,['Mr. Amyand'],['#Mr._Amyand'],['East Indies'],['#East_Indies'],['1799']
1,Miscellanies.,['Mr. Theobalds'],['#James_Theobalds_Esqr'],['Norway'],['#Norway'],['1801']
2,Miscellanies.,['Mr. Roberts'],['#Mr_Roberts'],[],[],['1806']
3,Miscellanies.,['Mr. Mc. Cormick'],['#MrMC'],['Lisbon'],['#Lisbon'],['1807']
4,Miscellanies.,[],[],['Lisbone'],['#Lisbon'],['1808']


In [16]:
# number of records
len(df)

1390

In [17]:
# records with no person. No of records with place name only.
df[df.person == '[]']


,div2_label,person,person_id,place,place_id,catnum
4,Miscellanies.,[],[],['Lisbone'],['#Lisbon'],['1808']
6,Miscellanies.,[],[],['China'],['#China'],['1810']
10,Miscellanies.,[],[],['Malabar'],['#Malabar'],['1818']
11,Miscellanies.,[],[],['England'],['#England'],['1819']
12,Miscellanies.,[],[],['London'],['#London'],['1']
...,...,...,...,...,...,...
1377,Agate cups botles spoons &c.,[],[],['East India'],['#East_Indies'],['194']
1380,Agate cups botles spoons &c.,[],[],['Suratte'],['#Suratte'],['203']
1383,Agate cups botles spoons &c.,[],[],['Germany'],['#Germany'],['218']
1387,Agate cups botles spoons &c.,[],[],['India'],['#India'],['239']


In [29]:

len(df[df.person == '[]'])

501

In [19]:
# records with no person or place
df[(df.person == '[]') & (df.place == "[]")]

,div2_label,person,person_id,place,place_id,catnum


In [20]:
# records with no place. Total no of records with person only.
len(df[df.place == "[]"])

546

In [21]:
# Both person and place are populated
len(df[(df.person != '[]') & (df.place != "[]")])

343

In [22]:
df[df.catnum == "['1823']"]

,div2_label,person,person_id,place,place_id,catnum
20,Miscellanies.,[],[],['Surinam'],['#Surinam'],['1823']


In [55]:
with open('people_and_places.csv', 'w') as file:
    writer = csv.writer(file)
    for row in people_csv_data:
        writer.writerow(row.values())

In [ ]:
#Identify if there is a most commonly occuring combination of person and place name?

In [23]:
def fn(place):
    place_list = eval(place)
    return len(place_list)

In [ ]:
count("['foo', 'bah']")

In [ ]:
df.head()

In [ ]:
df_place_populated = df[df.place != '[]']
#  846 rows
df_place_populated['place_numbers'] = df_place_populated.apply(lambda x: fn(x['place']), axis=1)

In [ ]:
df_place_populated.place_numbers.sum()

In [ ]:
df_place_populated

In [ ]:
df_place_populated.place

In [27]:
df['person_count'] = df.apply(lambda x: fn(x['person']), axis=1)
df['place_count'] = df.apply(lambda x: fn(x['place']), axis=1)

In [26]:
#total number of person name (count every person element)
df.person_count.sum()

1044

In [28]:
#total number of place name (count every place name element)
df.place_count.sum()

970